- Krigagem simples --> GP com média conhecida e função de autocovariância

- Krigagem ordinária --> GP com média constante desconhecida , função de autocovariância conhecida

- Krigagem Universal --> GP com média desconhecida da forma $h(x)^⊤β$ , função de autocovariância conhecida

-  A GP é baseada na Máxima Verossimilhança e na distribuição condicional, enquanto a Krigagem utiliza estimativa pontual e minimização com multiplicadores de Lagrange, resultando no que é conhecido em estatística como estimador BLUP

- Regression is used to find a function that represents a set of data points as closely as possible
- A  [Gaussian process](https://distill.pub/2019/visual-exploration-gaussian-processes/) is a probabilistic method that gives a confidence for the predicted function

In [45]:
# === IMPORTS ===
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ConstantKernel as C
from scipy.spatial import Delaunay
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.offline as py

# Author

$f(x_1,x_2) = 1.3356 \cdot (1.5 \cdot (1 - x_1)) + e^{2x_1 - 1} \cdot \sin(3\pi (x_1 - 0.6)^2) + e^{3x_2 - 0.5} \cdot \sin(4\pi (x_2 - 0.9)^2)$

In [ ]:
def f(x1, x2):
    return (1.3356 * (1.5 * (1 - x1)) 
            + np.exp((2 * x1) - 1) * np.sin((3 * np.pi) * ((x1 - 0.6) ** 2))
            + np.exp(3 * (x2 - 0.5)) * np.sin((4 * np.pi) * ((x2 - 0.9) ** 2)))

# Peaks


$g(x_1, x_2) = 3 \cdot (1-x_1) ^2  \cdot  e^{(-(x_1 ^2) - (x_2+1) ^2)} - 10 \cdot (x_1/5 - x_1 ^3 - x_2 ^5)  \cdot  e^{(-x_1 ^2 - x_2 ^2)} - 1/3  \cdot  e^{(-(x_1+1) ^2 - x_2 ^2)}$

In [ ]:
def g(x1, x2):    
    return(3*(1-x1)**2 * np.exp(-(x1**2) - (x2+1)**2)
           - 10*(x1/5 - x1**3 - x2**5) * np.exp(-x1**2 - x2**2)
           - 1/3 * np.exp(-(x1+1)**2 - x2**2))

# Métricas

In [48]:
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_percentage_error 

def ComputeMetrics(model, df_train, df_test, df_1000, df_vs, func):
    test_pred  = model.execute('points', df_test['x1'], df_test['x2'])[0]
    sup_pred   = model.execute('points', df_1000['x1'], df_1000['x2'])[0]
    orig_vs    = func(df_vs["x1"], df_vs["x2"])
    
    return {
        'r2_sup': r2_score(df_1000["f(x1,x2)"], sup_pred),
        'mse_sup': mean_squared_error(df_1000["f(x1,x2)"], sup_pred),
        'mape_sup': mean_absolute_percentage_error(df_1000["f(x1,x2)"], sup_pred),
        'rmse_sup': root_mean_squared_error(df_1000["f(x1,x2)"], sup_pred),
        'r2_test': r2_score(df_test['f(x1,x2)'], test_pred),
        'mse_test': mean_squared_error(df_test['f(x1,x2)'], test_pred),
        'mape_test': mean_absolute_percentage_error(df_test['f(x1,x2)'], test_pred),
        'rmse_test': root_mean_squared_error(df_test['f(x1,x2)'], test_pred),
        'r2_vs': r2_score(orig_vs, df_vs['f(x1,x2)']),
        'mse_vs': mean_squared_error(orig_vs, df_vs['f(x1,x2)']),
        'mape_vs': mean_absolute_percentage_error(orig_vs, df_vs['f(x1,x2)']),
        'rmse_vs': root_mean_squared_error(orig_vs, df_vs['f(x1,x2)']),
    }

# Visualização

In [49]:
def PrintInputSpace(original_samples_x, virtual_samples_x, df_test, tag):
    fig = go.Figure()

    # Limites dos eixos com margem
    x1_min, x1_max = original_samples_x[:, 0].min(), original_samples_x[:, 0].max()
    x2_min, x2_max = original_samples_x[:, 1].min(), original_samples_x[:, 1].max()

    margin = 0.05  # margem visual
    x1_range = [x1_min - margin, x1_max + margin]
    x2_range = [x2_min - margin, x2_max + margin]

    # Amostras originais
    fig.add_trace(go.Scatter(
        x=original_samples_x[:, 0],
        y=original_samples_x[:, 1],
        mode='markers',
        name='Amostras Originais',
        marker=dict(color='gray', size=7, opacity=0.8, symbol='circle')
    ))

    # Amostras virtuais
    fig.add_trace(go.Scatter(
        x=virtual_samples_x[:, 0],
        y=virtual_samples_x[:, 1],
        mode='markers',
        name='Amostras Virtuais',
        marker=dict(color='blue', size=7, opacity=0.8, symbol='circle')
    ))

    # Amostras de teste
    fig.add_trace(go.Scatter(
        x=df_test['x1'].values,
        y=df_test['x2'].values,
        mode='markers',
        name='Amostras de Teste',
        marker=dict(color='darkgreen', size=7, opacity=0.8, symbol='circle')
    ))

    fig.update_layout(
        title='Projeção bidimensional das amostras originais e virtuais',
        xaxis=dict(
            title='x1',
            range=x1_range,
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgray',
            zeroline=False
        ),
        yaxis=dict(
            title='x2',
            range=x2_range,
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgray',
            zeroline=False
        ),
        legend=dict(
            orientation='h',     # horizontal
            x=0.5,
            y=-0.2,
            xanchor='center',
            yanchor='top',
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='lightgray',
            borderwidth=1
        ),
        template='plotly_white',
        width=800,
        height=600,
        margin=dict(b=100)  # espaço extra na base
    )
    fig.write_image(f"./results_{tag}/figures/InputSpace.pdf", format='pdf')


In [50]:
def PlotSurface(Gpr, model_name, mesh_x=32, mesh_y=32):
    from plotly.subplots import make_subplots

    # Dados
    X = Gpr.df_1000['x1'].values
    Y = Gpr.df_1000['x2'].values
    Z = Gpr.y_pred_surface
    x_sup, y_sup, z_sup = Gpr.df_1000['x1'], Gpr.df_1000['x2'], Gpr.df_1000['f(x1,x2)']
    x_orig, y_orig, z_orig = Gpr.df_train['x1'], Gpr.df_train['x2'], Gpr.df_train['f(x1,x2)']
    
    points = np.column_stack((x_sup, y_sup))
    tri = Delaunay(points)
    
    fig_surface = ff.create_trisurf(x=x_sup, y=y_sup, z=z_sup, simplices=tri.simplices, colormap='Greys')

    # Métricas
    metrics = Gpr.metrics

    latex_map = {
        'r2_test':        'R²_test',
        'mse_test':       'MSE_test',
        'rmse_test':      'RMSE_test',
        'mape_test':      'MAPE_test',
        'r2_sup':         'R²_sup',
        'mse_sup':        'MSE_sup',
        'rmse_sup':       'RMSE_sup',
        'mape_sup':       'MAPE_sup',
        'r2_vs':          'R²_vs',
        'mse_vs':         'MSE_vs',
        'rmse_vs':        'RMSE_vs',
        'mape_vs':        'MAPE_vs',
    }

    metric_labels = [latex_map.get(k, k) for k in metrics.keys()]
    metric_values = [f"{v:.4f}" for v in metrics.values()]

    # Subplot com gráfico 3D e tabela
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'scene'}, {'type': 'table'}]],
        column_widths=[0.75, 0.25],
        horizontal_spacing=0.05
    )

    # Adicionar a superfície original como uma superfície
    fig.add_trace(fig_surface.data[0])

    # Superfície predita
    fig.add_trace(go.Surface(
        x=X.reshape(mesh_x, mesh_y),
        y=Y.reshape(mesh_x, mesh_y),
        z=Z,
        colorscale="ice",
        showscale=False,
        name='Predição'
    ), row=1, col=1)

    # Amostras originais
    fig.add_trace(go.Scatter3d(
        x=x_orig, y=y_orig, z=z_orig,
        mode='markers',
        marker=dict(size=5, color='gray', opacity=0.8),
        name='Amostras originais'
    ), row=1, col=1)

    # Amostras virtuais
    fig.add_trace(go.Scatter3d(
        x=Gpr.virtual_samples_df['x1'],
        y=Gpr.virtual_samples_df['x2'],
        z=Gpr.virtual_samples_df['f(x1,x2)'],
        mode='markers',
        marker=dict(size=5, color='blue', opacity=0.8),
        name='Amostras virtuais'
    ), row=1, col=1)

    # Amostras de teste
    fig.add_trace(go.Scatter3d(
        x=Gpr.df_test['x1'],
        y=Gpr.df_test['x2'],
        z=Gpr.df_test['f(x1,x2)'],
        mode='markers',
        marker=dict(size=5, color='darkgreen', opacity=0.8),
        name='Amostras de teste'
    ), row=1, col=1)

    # Tabela com métricas
    fig.add_trace(go.Table(
        header=dict(
            values=['<b>Métrica</b>', '<b>Valor</b>'],
            fill_color='royalblue',
            align='left',
            font=dict(color='white', size=12)
        ),
        cells=dict(
            values=[metric_labels, metric_values],
            fill_color=[['paleturquoise', 'white'] * ((len(metrics) // 2) + 1)],
            align='left',
            font=dict(size=12)
        )
    ), row=1, col=2)

    # Layout final com legenda abaixo
    fig.update_layout(
        title=f'Superfície e Métricas — {model_name}',
        scene=dict(
            xaxis_title='x1',
            yaxis_title='x2',
            zaxis_title='f(x1, x2)',
            aspectmode='cube',
            xaxis=dict(backgroundcolor="white"),
            yaxis=dict(backgroundcolor="white"),
            zaxis=dict(backgroundcolor="white")
        ),
        height=800,
        width=1100,
        margin=dict(t=50, b=120, l=20, r=20),
        legend=dict(
            orientation='h',
            x=0.5,
            xanchor='center',
            y=-0.2,
            yanchor='middle',
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='lightgray',
            borderwidth=1
        )
    )
    
    fig.write_image(f"./results_{Gpr.tag}/figures/{model_name}_surface.pdf", format='pdf')


# Extraindo dados

In [51]:
def getData(grid, func, max, min):
    x1 = np.linspace(min, max, grid)
    x2 = np.linspace(min, max, grid)
    x1, x2 = np.meshgrid(x1, x2)
    f_x1_x2 = func(x1, x2)
    return pd.DataFrame({'x1': x1.flatten(), 'x2': x2.flatten(), 'f(x1,x2)': f_x1_x2.flatten()})

In [52]:
class GPRVSG:
    def __init__(self, func, min, max, tag):
        self.func = func
        self.tag = tag
        self.df_train = getData(5, func, min, max)
        self.df_test     = getData(4, func, min, max)
        self.df_1000     = getData(32, func, min, max)

        self.original_samples_x  = self.df_train[['x1', 'x2']].values
        self.original_samples_y  = self.df_train['f(x1,x2)'].values
        self.d = self.original_samples_x.shape[1]

        self.virtual_samples_x = []
        self.virtual_samples_y = []
        self.SetInputSpace()


    def ComputeProjection(self):
        projections = []
        for m in range(self.d):
            x = self.original_samples_x[:, m]
            v = self.original_samples_x[:, m]
            xm_proj = (np.dot(x, v) / np.linalg.norm(v) ** 2) * v
            projections.append(np.sort(xm_proj))
        return projections

    def SetInputSpace(self):
        projections = self.ComputeProjection()
        avg_dists = [np.mean(np.diff(proj)) for proj in projections]

        Q_alpha = np.quantile(projections, [0.025, 0.25, 0.5, 0.75, 0.975], axis=1, method="hazen").T
        for m in range(self.d):
            for i in range(len(projections[m]) - 1):
                dist = projections[m][i + 1] - projections[m][i]
                if dist > avg_dists[m]:
                    G = (projections[m][i] + projections[m][i + 1]) / 2
                    for q in range(self.d):
                        if q != m:
                            for quantile in Q_alpha[q]:
                                tilde_q = np.zeros(self.d)
                                tilde_q[m] = G
                                tilde_q[q] = quantile
                                self.virtual_samples_x.append(tilde_q)
        self.virtual_samples_x = np.array(self.virtual_samples_x)

    def getX(self):
        df = pd.DataFrame({"x1": self.virtual_samples_x[:, 0],
                           "x2": self.virtual_samples_x[:, 1]})
        return df["x1"], df["x2"]

    def ComputeY(self, kernel, model_name):
        self.kernel = kernel
        self.model_name = model_name
        self.BuildModel()

        x1, x2 = self.getX()
        X = np.column_stack((x1, x2))
        self.virtual_samples_y = self.model.predict(X)
        self.virtual_samples_df = pd.DataFrame({
            "x1": x1,
            "x2": x2,
            "f(x1,x2)": self.virtual_samples_y
        })
        return self.virtual_samples_df

    def execute(self, mode, x1, x2):
        if mode != 'points':
            raise ValueError("Modo não suportado. Use 'points'.")
        X = np.column_stack((x1, x2))
        y_pred = self.model.predict(X)
        return y_pred, None

    def BuildModel(self, grid_pred=32):
        X_train = self.df_train[['x1', 'x2']].values
        y_train = self.df_train['f(x1,x2)'].values
        self.model = GaussianProcessRegressor(kernel=self.kernel, n_restarts_optimizer=100,
                                              alpha=1e-8, normalize_y=True)
        self.model.fit(X_train, y_train)

        X_pred = self.df_1000[['x1', 'x2']].values
        y_pred = self.model.predict(X_pred)
        self.y_pred_surface = y_pred.reshape(grid_pred, grid_pred)

    def Results(self, grid_pred=32):
        self.metrics = ComputeMetrics(self, self.df_train, self.df_test, self.df_1000, self.virtual_samples_df, self.func)

        # Visualização 3D
        PlotSurface(
            self,
            model_name=self.model_name,
            mesh_x=grid_pred,
            mesh_y=grid_pred
        )

In [53]:
import os
import pandas as pd
from sklearn.gaussian_process.kernels import (
    RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, WhiteKernel, ConstantKernel as C
)

# Lista de kernels a serem testados
kernel_list = [
    ("RBF",                   C(1.0) * RBF(length_scale=1.0, length_scale_bounds=(1e-6, 1e+6))),
    ("Matern_0.5",            C(1.0) * Matern(length_scale=1.0, nu=0.5, length_scale_bounds=(1e-6, 1e+6))),
    ("RationalQuadratic",     C(1.0) * RationalQuadratic(length_scale=1.0, alpha=0.1, length_scale_bounds=(1e-6, 1e+6))),
    ("ExpSineSquared",        C(1.0) * ExpSineSquared(length_scale=1.0, periodicity=3.0, length_scale_bounds=(1e-6, 1e+6))),
    ("DotProduct",            C(1.0) * DotProduct() + WhiteKernel())
]

# Função para testar kernelsfg e exportar resultados
def test_kernels(func, x_min, x_max, tag="f"):
    output_dir = f"results_{tag}"
    os.makedirs(output_dir, exist_ok=True)
    metrics_all = []
    gpr_vsg = GPRVSG(func, x_min, x_max, tag)
    PrintInputSpace(gpr_vsg.original_samples_x, gpr_vsg.virtual_samples_x, gpr_vsg.df_test, tag)
    for name, kernel in kernel_list:
        print(f"\n===== Testando kernel: {name} =====")

        gpr_vsg.ComputeY(kernel, name)
        gpr_vsg.Results(grid_pred=32)

        # Exportar amostras virtuais
        vs_filename = os.path.join(output_dir, f"virtual_samples_{name}.xlsx")
        gpr_vsg.virtual_samples_df.to_excel(vs_filename, index=False)

        gpr_vsg.metrics["Kernel"] = name
        metrics_all.append(gpr_vsg.metrics)

    # Exportar todas as métricas
    df_metrics = pd.DataFrame(metrics_all)
    metrics_filename = os.path.join(output_dir, "all_metrics.xlsx")
    df_metrics.to_excel(metrics_filename, index=False)

In [54]:
test_kernels(g, -2, 2, tag="g")


===== Testando kernel: RBF =====

===== Testando kernel: Matern_0.5 =====

===== Testando kernel: RationalQuadratic =====


/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k2__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.




===== Testando kernel: ExpSineSquared =====

===== Testando kernel: DotProduct =====


/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k2__sigma_0 is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.



In [55]:
test_kernels(f, 0, 1, tag="f")


===== Testando kernel: RBF =====

===== Testando kernel: Matern_0.5 =====

===== Testando kernel: RationalQuadratic =====


/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k2__alpha is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.




===== Testando kernel: ExpSineSquared =====

===== Testando kernel: DotProduct =====


/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

/home/codespace/.local/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k2__sigma_0 is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

